In [10]:
#Importando bibliotecas

import numpy as np

import torch
import torch.nn as nn
    
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from torch.utils.data import DataLoader

from pathlib import Path
from typing import List, Tuple

import numpy as np
import pandas as pd
import torch
import pytorch_lightning as pl
from torch.utils.data import DataLoader
import torch.nn as nn
import plotly.graph_objects as go

In [11]:
DATASET_NAME = 'UCI'

# CPC SSL Training

In [12]:
# Dados de Treino

data_path = Path(f'/workspaces/betania/ssl_tools/view_concatenated/{DATASET_NAME}/train')

datas_x_train = []

data_y_train = []

for f in data_path.glob('*.csv'):
    data = pd.read_csv(f)
    x = data[['accel-x', 'accel-y', 'accel-z', 'gyro-x', 'gyro-y', 'gyro-z']].values

    # Expend dimension

    x = np.swapaxes(x, 1, 0)
    
    datas_x_train.append(x)

    y = data['standard activity code'].values

    data_y_train.append(y)



In [13]:
# Dados de Teste

data_path = Path(f'/workspaces/betania/ssl_tools/view_concatenated/{DATASET_NAME}/test')

datas_x_test = []

data_y_test = []

for f in data_path.glob('*.csv'):
    data = pd.read_csv(f)
    x = data[['accel-x', 'accel-y', 'accel-z', 'gyro-x', 'gyro-y', 'gyro-z']].values

    # Expend dimension

    x = np.swapaxes(x, 1, 0)
    
    datas_x_test.append(x)

    y = data['standard activity code'].values

    data_y_test.append(y)

In [14]:
x_train = datas_x_train
y_train = data_y_train
x_test = datas_x_test
y_test = data_y_test


In [15]:
import torch
import torch.nn as nn

class CNNEncoder(nn.Module):
    def __init__(self, num_channels, kernel_size=3, dropout_rate=0.2, device: str = "cpu"):
        super(CNNEncoder, self).__init__()
        self.device = device

        self.conv1 = nn.Sequential(
            nn.Conv1d(num_channels, 32, kernel_size, padding=1),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )
        self.conv2 = nn.Sequential(
            nn.Conv1d(32, 64, kernel_size, padding=1),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )
        self.conv3 = nn.Sequential(
            nn.Conv1d(64, 128, kernel_size, padding=1),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )

    def forward(self, x):
        x = x.to(self.device)  # Mova os dados para a GPU, se disponível
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        return x


In [16]:
class CPC(pl.LightningModule):
    def __init__(
        self,
        encoder: torch.nn.Module,
        density_estimator: torch.nn.Module,
        auto_regressor: torch.nn.Module,
        lr: float = 1e-3,
        weight_decay: float = 0.0,
        window_size: int = 4,
        n_size: int = 5
    ):
        super().__init__()
        self.encoder = encoder.to(self.device)
        self.density_estimator = density_estimator.to(self.device)
        self.auto_regressor = auto_regressor.to(self.device)
        self.learning_rate = lr
        self.weight_decay = weight_decay
        self.window_size = window_size
        self.n_size = n_size
        self.training_step_losses = []
        self.best_train_loss = float('inf')


    def training_step(self, batch, batch_idx):
        assert len(batch) == 1, "Batch must be 1 sample only"
        sample = batch
        sample = sample.squeeze(0)
        rnd_t = np.random.randint(
            5 * self.window_size, sample.shape[-1] - 5 * self.window_size
        )
        sample = torch.tensor(
            sample[
                :,
                max(0, (rnd_t - 20 * self.window_size)) : min(
                    sample.shape[-1], rnd_t + 20 * self.window_size
                ),
            ]
        ).cpu()

        T = sample.shape[-1]
        windowed_sample = np.split(
            sample[:, : (T // self.window_size) * self.window_size],
            (T // self.window_size),
            -1,
        )
        windowed_sample = torch.tensor(np.stack(windowed_sample, 0), device=self.device)
        encodings = self.encoder(windowed_sample)
        window_ind = torch.randint(2, len(encodings) - 2, size=(1,))
        _, c_t = self.auto_regressor(
            encodings[max(0, window_ind[0] - 10) : window_ind[0] + 1].unsqueeze(0)
        )
        density_ratios = torch.bmm(
            encodings.unsqueeze(1),
            self.density_estimator(c_t.squeeze(1).squeeze(0)).expand_as(encodings).unsqueeze(-1),
        ).view(
            -1,
        )
        r = set(range(0, window_ind[0] - 2))
        r.update(set(range(window_ind[0] + 3, len(encodings))))
        rnd_n = np.random.choice(list(r), self.n_size)
        X_N = torch.cat(
            [density_ratios[rnd_n], density_ratios[window_ind[0] + 1].unsqueeze(0)], 0
        )
        labels = torch.Tensor([len(X_N) - 1]).to(self.device)
        loss = torch.nn.CrossEntropyLoss()(X_N.view(1, -1), labels.long())
        if loss < self.best_train_loss:
            self.best_train_loss = loss
        self.training_step_losses.append(loss)
        return loss


    def on_train_epoch_end(self) -> None:
        # do something with all training_step outputs, for example:
        epoch_mean = torch.stack(self.training_step_losses).mean()
        self.log("train_loss", epoch_mean, on_epoch=True, on_step=False, prog_bar=True, logger=True)
        if epoch_mean < self.best_train_loss:
            self.best_train_loss = epoch_mean
            self.log("best_train_loss", self.best_train_loss, on_epoch=True, on_step=False, prog_bar=False, logger=True)
        # free up the memory
        self.training_step_losses.clear()
        
    def configure_optimizers(self):
        learnable_parameters = (
            list(self.density_estimator.parameters())
            + list(self.encoder.parameters())
            + list(self.auto_regressor.parameters())
        )

        optimizer = torch.optim.Adam(
            learnable_parameters, lr=self.learning_rate, weight_decay=self.weight_decay
        )
        return optimizer

In [17]:
# Crie uma instância do ModelCheckpoint para salvar nos pontos de verificação correspondentes à perda mínima
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    filename='best_model',
    monitor='train_loss',
    mode='min',
    save_top_k=1,
    save_last=False,
    verbose=True
)

In [18]:
in_channel = 6
encoding_size = 10

encoder = CNNEncoder(in_channel=6, encoding_size = 10, device='cuda')
density_estimator = torch.nn.Linear(encoding_size, encoding_size)
auto_regressor = torch.nn.GRU(
    input_size=encoding_size, 
    hidden_size=encoding_size, 
    batch_first=True
)

cpc = CPC(encoder, density_estimator, auto_regressor)

TypeError: CNNEncoder.__init__() got an unexpected keyword argument 'in_channel'

In [ ]:
class SimpleDataset:
    def __init__(self, X, y=None):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        if self.y is not None:
            return self.X[idx].astype(np.float32), self.y[idx].astype(np.float32)
        else:
            return self.X[idx].astype(np.float32)
    
train_dataset = SimpleDataset(x_train)
test_dataset = SimpleDataset(x_test, y_test)
len(train_dataset), len(test_dataset)

(21, 6)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=1, num_workers=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, num_workers=10, shuffle=False)

In [ ]:
trainer = pl.Trainer(max_epochs=500, accelerator="gpu", devices=1, callbacks=[checkpoint_callback])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(cpc, train_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name              | Type       | Params
-------------------------------------------------
0 | encoder           | GRUEncoder | 66.8 K
1 | density_estimator | Linear     | 110   
2 | auto_regressor    | GRU        | 660   
-------------------------------------------------
67.6 K    Trainable params
0         Non-trainable params
67.6 K    Total params
0.270     Total estimated model params size (MB)
/home/vscode/.local/lib/python3.10/site-packages/pytorch_lightning/loops/fit_loop.py:293: The number of training batches (21) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0: 100%|██████████| 21/21 [00:00<00:00, 125.15it/s, v_num=290]

/tmp/ipykernel_2881691/2124898713.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  sample = torch.tensor(
Epoch 0, global step 21: 'train_loss' reached 1.76650 (best 1.76650), saving model to '/workspaces/betania/ssl_tools/cpc/lightning_logs/version_290/checkpoints/best_model.ckpt' as top 1


Epoch 1:   0%|          | 0/21 [00:00<?, ?it/s, v_num=290]          

Epoch 1: 100%|██████████| 21/21 [00:00<00:00, 39.83it/s, v_num=290, train_loss=1.770]

...

Epoch 498: 100%|██████████| 21/21 [00:00<00:00, 32.36it/s, v_num=290, train_loss=1.250]

Epoch 498, global step 10479: 'train_loss' was not in top 1


Epoch 499: 100%|██████████| 21/21 [00:00<00:00, 37.92it/s, v_num=290, train_loss=1.170]

Epoch 499, global step 10500: 'train_loss' was not in top 1
`Trainer.fit` stopped: `max_epochs=500` reached.


Epoch 499: 100%|██████████| 21/21 [00:00<00:00, 36.63it/s, v_num=290, train_loss=1.170]


In [ ]:
import os
from pathlib import Path

# Diretório onde os checkpoints estão salvos
checkpoint_dir = '/workspaces/betania/ssl_tools/cpc/lightning_logs/'

# Lista todas as versões no diretório
all_versions = [d for d in os.listdir(checkpoint_dir) if os.path.isdir(os.path.join(checkpoint_dir, d)) and d.startswith('version_')]

# Ordena as versões em ordem decrescente
sorted_versions = sorted(all_versions, key=lambda x: int(x.split('_')[1]), reverse=True)

# Obtém a versão mais recente
latest_version = sorted_versions[0]

VERSION = latest_version

# Crie uma instância do modelo

cpc = CPC(encoder, density_estimator, auto_regressor)

# Carregue os pesos salvos no ponto de verificação 'best_model.ckpt' para o cpc

# Mudando a versão do checkpoint

checkpoint_path = f'/workspaces/betania/ssl_tools/cpc/lightning_logs/{VERSION}/checkpoints/best_model.ckpt'
checkpoint = torch.load(checkpoint_path, map_location='cuda' if torch.cuda.is_available() else 'cpu')

# Carregue os pesos no seu modelo
cpc.load_state_dict(checkpoint['state_dict'])




<All keys matched successfully>

In [ ]:
cpc.eval()

CPC(
  (encoder): GRUEncoder(
    (rnn): GRU(6, 100, bidirectional=True)
    (nn): Linear(in_features=200, out_features=10, bias=True)
  )
  (density_estimator): Linear(in_features=10, out_features=10, bias=True)
  (auto_regressor): GRU(10, 10, batch_first=True)
)

## CPC Fine-Tuning

We are going to fine-tune the CPC model on the downstream task of classification. We will use the same dataset and re-use the same encoder.

In [ ]:
from typing import Any, Optional

from torchmetrics.functional import accuracy

# Classificando com uma MLP

class StateClassifier(torch.nn.Module):
    def __init__(self, input_size: int= 10, hidden_size1= 64, hidden_size2=64, num_classes= 7, dropout_prob= 0):
        super(StateClassifier, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(input_size, hidden_size1),
            nn.ReLU()
        )
        self.layer2 = nn.Sequential(
            nn.Linear(hidden_size1, hidden_size2),
            nn.ReLU()
        )
        self.dropout = nn.Dropout(p=dropout_prob)
        self.output_layer = nn.Sequential(
            nn.Linear(hidden_size2, num_classes),
            nn.Softmax(dim=1)
        )
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.dropout(out)
        out = self.output_layer(out)
        return out

 #Linear
    
# class StateClassifier(torch.nn.Module):
#     def __init__(self, input_size: int = 10, n_classes: int = 7):
#         super(StateClassifier, self).__init__()
#         self.input_size = input_size
#         self.n_classes = n_classes
#         self.normalize = torch.nn.BatchNorm1d(self.input_size)
#         self.nn = torch.nn.Linear(self.input_size, self.n_classes)
#         torch.nn.init.xavier_uniform_(self.nn.weight)

#     def forward(self, x):
#         x = self.normalize(x)
#         logits = self.nn(x)
#         return logits

class CPC_Classifier(pl.LightningModule):
    def __init__(
        self,
        encoder: torch.nn.Module,
        classifier: torch.nn.Module,
        lr: float = 1e-3,
        weight_decay: float = 0.0,
        task_class: str = "multiclass",
        num_classes: int = 7
    ):
        super().__init__()
        self.encoder = encoder.to(self.device)
        self.classifier = classifier.to(self.device)
        self.learning_rate = lr
        self.weight_decay = weight_decay
        self.training_step_losses = []
        self.validation_step_losses = []
        self.loss_function = torch.nn.CrossEntropyLoss()
        self.task_class = task_class
        self.num_classes = num_classes
        self.best_validation_loss = float('inf')
        
    def configure_optimizers(self) -> Any:
        optimizer = torch.optim.Adam(
            self.classifier.parameters(), lr=self.learning_rate, weight_decay=self.weight_decay
        )
        return optimizer
    
    def forward(self, x):
        encodings = self.encoder(x)
        predictions = self.classifier(encodings)
        return predictions
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        predictions = self.forward(x)
        loss = self.loss_function(predictions, y.long())
        self.training_step_losses.append(loss)
        return loss
    
    def on_train_epoch_end(self) -> None:
        # do something with all training_step outputs, for example:
        epoch_mean = torch.stack(self.training_step_losses).mean()
        self.log("train_loss", epoch_mean, on_epoch=True, on_step=False, prog_bar=True, logger=True)
        # free up the memory
        self.training_step_losses.clear()
    
    def validation_step(self, batch, batch_idx):
        loss, acc = self._shared_eval_step(batch, batch_idx)
        if loss < self.best_validation_loss:
            self.best_validation_loss = loss
        self.validation_step_losses.append(loss)
        metrics = {"val_acc": acc, "val_loss": loss}
        self.log_dict(metrics)
        return metrics
    
    def test_step(self, batch, batch_idx):
        loss, acc = self._shared_eval_step(batch, batch_idx)
        metrics = {"test_acc": acc, "test_loss": loss}
        self.log_dict(metrics)
        return metrics

        
    def on_validation_epoch_end(self) -> None:
        # do something with all training_step outputs, for example:
        epoch_mean = torch.stack(self.validation_step_losses).mean()
        self.log("val_loss", epoch_mean, on_epoch=True, on_step=False, prog_bar=True, logger=True)
        if epoch_mean < self.best_validation_loss:
            self.best_validation_loss = epoch_mean
            self.log("best_validation_loss", self.best_validation_loss, on_epoch=True, on_step=False, prog_bar=False, logger=True)
        # free up the memory
        self.validation_step_losses.clear()

    def _shared_eval_step(self, batch, batch_idx):
        x, y = batch
        predictions = self.forward(x)
        loss = self.loss_function(predictions, y.long())
        acc = accuracy(torch.argmax(predictions, dim=1), y.long(), task=self.task_class, num_classes=self.num_classes)
        return loss, acc

In [ ]:
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    filename='best_model',
    monitor='val_loss',  # escolha a métrica que você deseja monitorar (pode ser 'val_acc' ou outra métrica)
    mode='min',  # 'min' significa que queremos minimizar a métrica monitorada (por exemplo, perda)
    save_top_k=1,  # salva apenas o melhor modelo
    save_last=False,  # salva o último modelo
    verbose=True
)

In [ ]:
import pandas as pd
import numpy as np
import torch
from pathlib import Path
from torch.utils.data import TensorDataset, DataLoader

BATCH_SIZE = 100

# Caminho dos dados

data_path_train = Path(f'/workspaces/betania/data/standartized_balanced/{DATASET_NAME}/train.csv')

data_path_validation = Path(f'/workspaces/betania/data/standartized_balanced/{DATASET_NAME}/validation.csv')

data_path_test = Path(f'/workspaces/betania/data/standartized_balanced/{DATASET_NAME}/test.csv')

# Train

x_train = pd.read_csv(data_path_train)

x_train = x_train.iloc[:, :360]
    
x_train = x_train.astype(np.float32)

y_train = pd.read_csv(data_path_train)

y_train = y_train.iloc[:, -1]

y_train = y_train.astype(np.float32)

tensor_x = torch.Tensor(np.array(x_train))

tensor_y = torch.Tensor(np.array(y_train))

original_dim = tensor_x.shape[0]

input_shape = (original_dim, 6, 60)

tensor_x = tensor_x.reshape(input_shape)

dataset_train= TensorDataset(tensor_x,tensor_y)

dataloader_train = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True, drop_last= True)

# Validation

x_validation = pd.read_csv(data_path_validation)

x_validation = x_validation.iloc[:, :360]

x_validation = x_validation.astype(np.float32)

y_validation = pd.read_csv(data_path_validation)

y_validation = y_validation.iloc[:, -1]

y_validation = y_validation.astype(np.float32)

tensor_x_val = torch.Tensor(np.array(x_validation))

tensor_y_val = torch.Tensor(np.array(y_validation))

original_dim_val = tensor_x_val.shape[0]

tensor_x_val = tensor_x_val.reshape(original_dim_val, 6, 60)

dataset_val= TensorDataset(tensor_x_val,tensor_y_val)

dataloader_val = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=True, drop_last = True)

# Test

x_test = pd.read_csv(data_path_test)

x_test = x_test.iloc[:, :360]

x_test = x_test.astype(np.float32)

y_test = pd.read_csv(data_path_test)

y_test = y_test.iloc[:, -1]

y_test = y_test.astype(np.float32)

tensor_x_test = torch.Tensor(np.array(x_test))

tensor_y_test = torch.Tensor(np.array(y_test))

original_dim_test = tensor_x_test.shape[0]

tensor_x_test = tensor_x_test.reshape((original_dim_test, 6, 60))

dataset_test= TensorDataset(tensor_x_test,tensor_y_test)

dataloader_test = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=True, drop_last= True)


In [ ]:
encoding_size = 10
n_classes = 7


#classifier = StateClassifier(input_size=encoding_size, n_classes=n_classes)

classifier = StateClassifier(input_size=encoding_size, num_classes=n_classes, dropout_prob=0, hidden_size1= 64, hidden_size2=32)
cpc_classifier = CPC_Classifier(encoder, classifier)

In [ ]:
cpc_classifier = cpc_classifier.to('cuda')

# Forward no conjunto de teste
with torch.no_grad():
    xtest = cpc_classifier.forward(tensor_x_test.to('cuda')).cpu().numpy()

# Forward no conjunto de treinamento
with torch.no_grad():
    xtrain = cpc_classifier.forward(tensor_x.to('cuda')).cpu().numpy()

# Converter rótulos para NumPy
ytrain = tensor_y.cpu().numpy()
ytest = tensor_y_test.cpu().numpy()


In [ ]:
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score

random_forest_model = RandomForestClassifier(n_estimators=100, random_state=42)

random_forest_model.fit(xtrain, ytrain)

# Faça previsões no conjunto de teste
predictions = random_forest_model.predict(xtest)

# Avalie o desempenho do modelo usando métricas, como a precisão
accuracy = accuracy_score(ytest, predictions)

print(f'A precisão do modelo no conjunto de teste é: {accuracy}')

A precisão do modelo no conjunto de teste é: 0.5579710144927537


In [ ]:
# KNN

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Criar o modelo KNN
knn_model = KNeighborsClassifier(n_neighbors=20)  # Pode ajustar o número de vizinhos conforme necessário

# Treinar o modelo
knn_model.fit(xtrain, ytrain)

# Fazer previsões no conjunto de teste
predictions_knn = knn_model.predict(xtest)

# Avaliar o desempenho do modelo usando métricas, como a precisão
accuracy_knn = accuracy_score(ytest, predictions_knn)

print("Accuracy (KNN):", accuracy_knn)

Accuracy (KNN): 0.5420289855072464


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Criar o modelo SVM
svm_model = SVC(kernel='linear', C=1.0)  # Pode ajustar o kernel e C conforme necessário

# Treinar o modelo
svm_model.fit(xtrain, ytrain)

# Fazer previsões no conjunto de teste
predictions_svm = svm_model.predict(xtest)

# Avaliar o desempenho do modelo usando métricas, como a precisão
accuracy_svm = accuracy_score(ytest, predictions_svm)

print("Accuracy (SVM):", accuracy_svm)

Accuracy (SVM): 0.3101449275362319


In [ ]:
#trainer = pl.Trainer(max_epochs=300, accelerator="gpu", devices=1, callbacks=[checkpoint_callback])

In [ ]:
#trainer.fit(cpc_classifier, train_dataloaders=dataloader_train, val_dataloaders=dataloader_val)

## Classification evaluation

In [ ]:
import os
from pathlib import Path

# Diretório onde os checkpoints estão salvos
checkpoint_dir = '/workspaces/betania/ssl_tools/cpc/lightning_logs/'

# Lista todas as versões no diretório
all_versions = [d for d in os.listdir(checkpoint_dir) if os.path.isdir(os.path.join(checkpoint_dir, d)) and d.startswith('version_')]

# Ordena as versões em ordem decrescente
sorted_versions = sorted(all_versions, key=lambda x: int(x.split('_')[1]), reverse=True)

# Obtém a versão mais recente
latest_version = sorted_versions[0]

VERSION = latest_version

# Carregue os pesos salvos no ponto de verificação 'best_model.ckpt' para o cpc

# Mudando a versão do checkpoint

checkpoint_path = f'/workspaces/betania/ssl_tools/cpc/lightning_logs/{VERSION}/checkpoints/best_model.ckpt'
checkpoint = torch.load(checkpoint_path, map_location='cuda' if torch.cuda.is_available() else 'cpu')

# Carregue os pesos no seu classificador
cpc_classifier.load_state_dict(checkpoint['state_dict'])


RuntimeError: Error(s) in loading state_dict for CPC_Classifier:
	Missing key(s) in state_dict: "classifier.layer1.0.weight", "classifier.layer1.0.bias", "classifier.layer2.0.weight", "classifier.layer2.0.bias", "classifier.output_layer.0.weight", "classifier.output_layer.0.bias". 
	Unexpected key(s) in state_dict: "density_estimator.weight", "density_estimator.bias", "auto_regressor.weight_ih_l0", "auto_regressor.weight_hh_l0", "auto_regressor.bias_ih_l0", "auto_regressor.bias_hh_l0". 

In [ ]:
trainer.test(cpc_classifier, dataloader_test)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]
/home/vscode/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:492: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
/home/vscode/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=127` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 6/6 [00:00<00:00, 75.09it/s] 
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc             0.621666669845581
        test_loss            1.544309377670288
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_acc': 0.621666669845581, 'test_loss': 1.544309377670288}]